In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os
import time
import random
from datetime import datetime


def augment_landmarks(landmarks, augmentation_factor=2):
    """
    Аугментация landmarks: случайное растяжение и небольшое вращение
    """
    augmented_samples = []

    for _ in range(augmentation_factor):
        augmented_landmarks = []

        for lm in landmarks:
            # Создаем копию landmark
            new_lm = type("", (), {})()  # Создаем пустой объект
            new_lm.x = lm.x
            new_lm.y = lm.y
            new_lm.z = lm.z

            # Случайное растяжение по осям (не слишком сильное)
            stretch_x = random.uniform(0.95, 1.05)
            stretch_y = random.uniform(0.95, 1.05)
            stretch_z = random.uniform(0.98, 1.02)  # По Z меньше изменений

            # Центр ладони для вращения
            center_x = np.mean([lm.x for lm in landmarks])
            center_y = np.mean([lm.y for lm in landmarks])

            # Небольшое вращение (максимум 15 градусов)
            angle = random.uniform(-15, 15) * np.pi / 180  # В радианах

            # Применяем растяжение
            stretched_x = (new_lm.x - center_x) * stretch_x + center_x
            stretched_y = (new_lm.y - center_y) * stretch_y + center_y
            stretched_z = new_lm.z * stretch_z

            # Применяем вращение только вокруг оси Z (не меняем ориентацию жеста)
            rotated_x = (
                center_x
                + (stretched_x - center_x) * np.cos(angle)
                - (stretched_y - center_y) * np.sin(angle)
            )
            rotated_y = (
                center_y
                + (stretched_x - center_x) * np.sin(angle)
                + (stretched_y - center_y) * np.cos(angle)
            )

            # Небольшое случайное смещение
            offset_x = random.uniform(-0.01, 0.01)
            offset_y = random.uniform(-0.01, 0.01)
            offset_z = random.uniform(-0.005, 0.005)

            # Финальные координаты
            new_lm.x = rotated_x + offset_x
            new_lm.y = rotated_y + offset_y
            new_lm.z = stretched_z + offset_z

            augmented_landmarks.append(new_lm)

        augmented_samples.append(augmented_landmarks)

    return augmented_samples


def extract_features_variations(landmarks):
    """
    Нормализация
    """

    try:
        # Нормализация относительно центра ладони
        center_x = np.mean([lm.x for lm in landmarks])
        center_y = np.mean([lm.y for lm in landmarks])
        center_z = np.mean([lm.z for lm in landmarks])

        distances = [
            np.linalg.norm([lm.x - center_x, lm.y - center_y, lm.z - center_z])
            for lm in landmarks
        ]
        scale = np.std(distances) if np.std(distances) > 1e-6 else 1e-6

        coords = []
        for lm in landmarks:
            coords.extend(
                [
                    (lm.x - center_x) / scale,
                    (lm.y - center_y) / scale,
                    (lm.z - center_z) / scale,
                ]
            )

        return coords

    except Exception as e:
        print(f"⚠️  Error in feature extraction: {e}")
        return None


def save_dataset(rows, output_csv, gesture_label):
    """
    Сохранение датасета с метаданными
    """
    columns = [f"{axis}{i}" for i in range(21) for axis in ("x", "y", "z")] + [
        "gesture",
        "is_right_hand",
    ]
    df = pd.DataFrame(rows, columns=columns)

    # Добавляем метаданные
    metadata = {
        "collection_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "gesture_label": gesture_label,
        "total_samples": len(df),
        "right_hand_samples": len(df[df["is_right_hand"] == 1]),
        "left_hand_samples": len(df[df["is_right_hand"] == 0]),
    }

    # Исправление: проверяем, есть ли путь для создания директории
    output_dir = os.path.dirname(output_csv)
    if output_dir:  # Если путь не пустой
        os.makedirs(output_dir, exist_ok=True)

    # Проверяем существующий файл и добавляем метаданные
    if os.path.exists(output_csv):
        df_existing = pd.read_csv(output_csv)
        df = pd.concat([df_existing, df], ignore_index=True)
        print(f"📁 Added to existing dataset. Total samples: {len(df)}")
    else:
        print(f"📄 Created new dataset with {len(df)} samples")

    df.to_csv(output_csv, index=False)

    # Сохраняем метаданные в отдельный файл
    meta_csv = output_csv.replace(".csv", "_metadata.txt")
    with open(meta_csv, "a") as f:
        f.write(f"\n=== Session {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} ===\n")
        for key, value in metadata.items():
            f.write(f"{key}: {value}\n")


def capture_hands_with_augmentation(
    output_csv,
    gesture_label,
    max_samples=100,
    augmentation_factor=2,  # Сколько аугментированных samples добавлять к каждому реальному
    capture_interval=0.3,
):
    """
    Улучшенный сбор данных с аугментацией жестов
    """
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils

    hands = mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=1,
        min_detection_confidence=0.7,
        min_tracking_confidence=0.5,
    )

    rows = []
    count = 0
    last_capture_time = 0
    augmentation_stats = {"original": 0, "augmented": 0}

    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("!!! Unable to access camera")
        return

    print(f"🎥 Capturing: {gesture_label} with augmentation")
    print(f"💡 Each sample will generate {augmentation_factor} augmented versions")
    print("💡 Move your hand around for better data variety!")

    while cap.isOpened() and count < max_samples:
        success, frame = cap.read()
        if not success:
            break

        frame_flipped = cv2.flip(frame, 1)
        image_rgb = cv2.cvtColor(frame_flipped, cv2.COLOR_BGR2RGB)
        result = hands.process(image_rgb)

        current_time = time.time()

        if (
            result.multi_hand_landmarks
            and result.multi_handedness
            and current_time - last_capture_time >= capture_interval
        ):

            hand_landmarks = result.multi_hand_landmarks[0]
            label = result.multi_handedness[0].classification[0].label
            is_right = 1 if label == "Right" else 0

            # ОРИГИНАЛЬНЫЙ sample
            original_coords = extract_features_variations(hand_landmarks.landmark)

            if original_coords is not None:
                # Добавляем оригинальный sample
                rows.append(original_coords + [gesture_label, is_right])
                augmentation_stats["original"] += 1
                count += 1

                print(f"✅ Sample {count}/{max_samples} | Hand: {label}")

                # АУГМЕНТИРОВАННЫЕ samples
                augmented_landmarks_list = augment_landmarks(
                    hand_landmarks.landmark, augmentation_factor
                )

                for i, aug_landmarks in enumerate(augmented_landmarks_list):
                    aug_coords = extract_features_variations(aug_landmarks)
                    if aug_coords is not None:
                        rows.append(aug_coords + [gesture_label, is_right])
                        augmentation_stats["augmented"] += 1
                        print(f"   ↳ Augmented {i+1}/{augmentation_factor}")

                last_capture_time = current_time

            # Рисуем landmarks
            mp_drawing.draw_landmarks(
                frame_flipped, hand_landmarks, mp_hands.HAND_CONNECTIONS
            )

        # Отображение информации
        info_text = [
            f"Gesture: {gesture_label}",
            f"Samples: {count}/{max_samples}",
            f"Augmented: +{augmentation_stats['augmented']}",
            "Move hand for variety!",
            "Press Q to quit",
        ]

        for i, text in enumerate(info_text):
            cv2.putText(
                frame_flipped,
                text,
                (10, 30 + i * 25),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0, 255, 0),
                2,
            )

        cv2.imshow("Hand Capture - WITH AUGMENTATION", frame_flipped)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()
    hands.close()

    if rows:
        save_dataset(rows, output_csv, gesture_label)
        print(f"🎉 Collection complete!")
        print(
            f"📊 Stats: {augmentation_stats['original']} original + {augmentation_stats['augmented']} augmented = {len(rows)} total samples"
        )

        # Анализ собранных данных
        analyze_collected_data(rows, gesture_label)
    else:
        print("❌ No samples captured.")


def visualize_augmentation_example(landmarks, augmented_landmarks):
    """
    Визуализация аугментации (для отладки)
    """
    # Создаем пустое изображение
    img = np.zeros((400, 600, 3), dtype=np.uint8)

    # Функция для преобразования координат в пиксели
    def to_pixels(x, y):
        return int((x + 0.5) * 400), int((y + 0.5) * 400)

    # Рисуем оригинальные landmarks (зеленые)
    for i, lm in enumerate(landmarks):
        px, py = to_pixels(lm.x, lm.y)
        cv2.circle(img, (px, py), 3, (0, 255, 0), -1)
        if i < len(landmarks) - 1:
            next_px, next_py = to_pixels(landmarks[i + 1].x, landmarks[i + 1].y)
            cv2.line(img, (px, py), (next_px, next_py), (0, 255, 0), 1)

    # Рисуем аугментированные landmarks (красные)
    for i, lm in enumerate(augmented_landmarks[0]):  # Первый аугментированный sample
        px, py = to_pixels(lm.x, lm.y)
        cv2.circle(img, (px, py), 3, (0, 0, 255), -1)
        if i < len(augmented_landmarks[0]) - 1:
            next_px, next_py = to_pixels(
                augmented_landmarks[0][i + 1].x, augmented_landmarks[0][i + 1].y
            )
            cv2.line(img, (px, py), (next_px, next_py), (0, 0, 255), 1)

    cv2.putText(
        img,
        "Green: Original, Red: Augmented",
        (10, 380),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        (255, 255, 255),
        1,
    )
    cv2.imshow("Augmentation Example", img)
    cv2.waitKey(1000)  # Показываем 1 секунду
    cv2.destroyWindow("Augmentation Example")


def analyze_collected_data(rows, gesture_label):
    """Быстрый анализ собранных данных"""
    print(f"\n📊 Analysis for {gesture_label}:")
    print(f"Total samples: {len(rows)}")

    # Анализ распределения рук
    right_hand_count = sum(1 for row in rows if row[-1] == 1)
    left_hand_count = len(rows) - right_hand_count
    print(f"Right hand: {right_hand_count}")
    print(f"Left hand: {left_hand_count}")

    # Анализ диапазона координат
    all_coords = [
        coord for row in rows for coord in row[:-2]
    ]  # Все координаты кроме меток
    print(f"Coordinate range: [{min(all_coords):.3f}, {max(all_coords):.3f}]")

    if right_hand_count == 0 or left_hand_count == 0:
        print("⚠️  Warning: Only one hand type detected. Try using both hands!")


# Пример использования
if __name__ == "__main__":
    print("🚀 Starting hand gesture data collection WITH AUGMENTATION...")

    # Захват данных с аугментацией
    capture_hands_with_augmentation(
        output_csv="don.csv",
        gesture_label="don",
        max_samples=300,  # Меньше оригинальных samples, но больше за счет аугментации
        augmentation_factor=2,  # 1 оригинальный + 2 аугментированных = 3 samples за раз
        capture_interval=0.1,
    )